<a href="https://colab.research.google.com/github/rbondili/Tweet_Sentiment_Analysis/blob/master/Tweet_Sentiment_Part_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align=center><font size=5>Tweet Sentiment Extraction Kaggel Competition</font></h1>

### Data <a id="data"></a>

#### About Tweet Sentiment details

"My ridiculous dog is amazing." [sentiment: positive]

With all of the tweets circulating every second it is hard to tell whether the sentiment behind a specific tweet will impact a company, or a person's, brand for being viral (positive), or devastate profit because it strikes a negative tone. Capturing sentiment in language is important in these times where decisions and reactions are created and updated in seconds. But, which words actually lead to the sentiment description? In this competition you will need to pick out the part of the tweet (word or phrase) that reflects the sentiment.

Help build your skills in this important area with this broad dataset of tweets. Work on your technique to grab a top spot in this competition. What words in tweets support a positive, negative, or neutral sentiment? How can you help make that determination using machine learning tools?

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd /content/drive/My\ Drive/Colab\ Notebooks/TweetSentiment/

/content/drive/My Drive/Colab Notebooks/TweetSentiment


In [0]:
ls

 sample_submission.csv  'Tweet Sentiment Extraction.ipynb'
 test.csv               'Tweet Sentiment Word Embedding Models.ipynb'
 train.csv


## Loading the Dataset

In [0]:
relative_path = '/content/drive/My Drive/Colab Notebooks/TweetSentiment/'

train = pd.read_csv(relative_path + 'train.csv')
test = pd.read_csv(relative_path + 'test.csv')

print(train.head())
print(test.head())

       textID  ... sentiment
0  cb774db0d1  ...   neutral
1  549e992a42  ...  negative
2  088c60f138  ...  negative
3  9642c003ef  ...  negative
4  358bd9e861  ...  negative

[5 rows x 4 columns]
       textID                                               text sentiment
0  f87dea47db  Last session of the day  http://twitpic.com/67ezh   neutral
1  96d74cb729   Shanghai is also really exciting (precisely -...  positive
2  eee518ae67  Recession hit Veronique Branquinho, she has to...  negative
3  01082688c6                                        happy bday!  positive
4  33987a8ee5             http://twitpic.com/4w75p - I like it!!  positive


In [0]:
X_train = train.drop('sentiment', axis=1)
y_train = train.sentiment
X_test = test.drop('sentiment', axis=1)
y_test = test.sentiment

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(27481, 3) (27481,) (3534, 2) (3534,)


In [0]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

import string
from nltk.corpus import stopwords
stopwords_list = stopwords.words('english')
# Add or remove any additional stopwords
stopwords_list.extend(['the'])
#stopwords_list.remove('no')
#stopwords_list.remove('not')

from nltk.tokenize import word_tokenize

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
def clean_text(text):
   
    # Convert text to all lowercase letters
    text = text.lower()
    
    # Remove whitespaces
    text = text.strip()
    
    # Tokenize text
    tokens = word_tokenize(text)
    
    # Remove all tokens that are not alphabetic   r[A-Za-z#]
    tokens = [token for token in tokens if token.isalpha()]
        
    # Remove stopwords
    tokens = [token for token in tokens if token not in stopwords_list]
    
    # Remove tokens less than three letters
    tokens = [token for token in tokens if len(token) > 2 ]
    
    # Remove punctuations
    tokens = [token for token in tokens if token not in string.punctuation]
    
    # Lemmetize tokens
    tokens = [ lemmatizer.lemmatize(token) for token in tokens ]
    
    # Stem tokens
    #tokens = 
    
    # Re-create text from filtered tokens, so that vectorizer won't complain
    text = ' '.join(tokens)
    return text

In [0]:
X_train.selected_text[0]

'I`d have responded, if I were going'

In [0]:
clean_text(X_train.text[0])

'responded going'

In [0]:
 X_train.text =  X_train.text.astype(str)
 X_test.text =  X_test.text.astype(str)

In [0]:
X_train_clean = X_train.text.apply(lambda x: clean_text(x))
X_test_clean = X_test.text.apply(lambda x: clean_text(x))

In [0]:
print(len(X_train), len(X_test), len(X_train_clean), len(X_test_clean))

27481 3534 27481 3534


In [0]:
X_train_clean[:5]

0                   responded going
1           sooo sad miss san diego
2                      bos bullying
3             interview leave alone
4    son put release already bought
Name: text, dtype: object

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()

# Training set
cv.fit(X_train_clean)
X_train_cv = cv.transform(X_train_clean)

# Test set
X_test_cv = cv.transform(X_test_clean)

In [0]:
X_train_cnt = pd.DataFrame(X_train_cv.todense(), columns=cv.get_feature_names())
X_train_cnt.head()

,aaa,aaaa,aaaaaaaaaaa,aaaaaaaaaahhhhhhhh,aaaaaaaaaamazing,aaaaaaaafternoon,aaaaaaaahhhhhhhh,aaaaaah,aaaaaahhhhhhhh,aaaaaawwwesome,aaaaahhhh,aaaaall,aaaaand,aaaaaoouoouoouu,aaaaarrrrggghhh,aaaaaw,aaaaawwwwwww,aaaagggessss,aaaagh,aaaah,aaaahh,aaaargh,aaaau,aaaaw,aaaawww,aaah,aaahaha,aaahh,aaahhh,aaand,aaargh,aaarrrggghhh,aaarrrgh,aaauuuggghhh,aaaw,aaaww,aaawww,aac,aaggh,aah,...,zenjar,zensify,zero,zeta,zic,zimmer,zion,zip,zipcode,zippys,zit,zoe,zoey,zoidberg,zoita,zojuist,zombie,zombiepix,zomg,zommgg,zone,zoned,zongzi,zoo,zoombezi,zoomed,zoozoo,zotz,zsg,zuccini,zulu,zum,zumba,zune,zwarte,zyrtec,zzzz,zzzzy,zzzzzzz,zzzzzzzzzzzzzzz
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
X_test_cnt = pd.DataFrame(X_test_cv.todense(), columns=cv.get_feature_names())
X_test_cnt.head()

,aaa,aaaa,aaaaaaaaaaa,aaaaaaaaaahhhhhhhh,aaaaaaaaaamazing,aaaaaaaafternoon,aaaaaaaahhhhhhhh,aaaaaah,aaaaaahhhhhhhh,aaaaaawwwesome,aaaaahhhh,aaaaall,aaaaand,aaaaaoouoouoouu,aaaaarrrrggghhh,aaaaaw,aaaaawwwwwww,aaaagggessss,aaaagh,aaaah,aaaahh,aaaargh,aaaau,aaaaw,aaaawww,aaah,aaahaha,aaahh,aaahhh,aaand,aaargh,aaarrrggghhh,aaarrrgh,aaauuuggghhh,aaaw,aaaww,aaawww,aac,aaggh,aah,...,zenjar,zensify,zero,zeta,zic,zimmer,zion,zip,zipcode,zippys,zit,zoe,zoey,zoidberg,zoita,zojuist,zombie,zombiepix,zomg,zommgg,zone,zoned,zongzi,zoo,zoombezi,zoomed,zoozoo,zotz,zsg,zuccini,zulu,zum,zumba,zune,zwarte,zyrtec,zzzz,zzzzy,zzzzzzz,zzzzzzzzzzzzzzz
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
from sklearn.preprocessing import LabelEncoder
out_encoder = LabelEncoder()
out_encoder.fit(y_train)
y_train_cnt = out_encoder.transform(y_train)
y_test_cnt = out_encoder.transform(y_test)

out_encoder.classes_

array(['negative', 'neutral', 'positive'], dtype=object)

In [0]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

naive_params = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
grid_naive = GridSearchCV(MultinomialNB(), naive_params)
grid_naive.fit(X_train_cnt, y_train_cnt)

# SVC best estimator
naive = grid_naive.best_estimator_
print(naive)

naive_score = cross_val_score(naive, X_train_cnt, y_train_cnt, cv=5)
print('Naive modeling using CountVectorizer Cross Validation Score: ', round(naive_score.mean() * 100, 2).astype(str) + '%')

y_test_pred =  naive.predict(X_test_cnt)
print("Test accuracy using CountVectorizer is:", round(accuracy_score(y_test_cnt,y_test_pred) * 100, 2).astype(str) + '%')

MultinomialNB(alpha=1, class_prior=None, fit_prior=True)
Naive modeling using CountVectorizer Cross Validation Score:  65.13%
Test accuracy using CountVectorizer is: 66.07%


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer()

# Training set
tf.fit(X_train_clean)
X_train_tf = tf.transform(X_train_clean)

# Test set
X_test_tf = tf.transform(X_test_clean)

X_train_tfid = pd.DataFrame(X_train_tf.todense(), columns=tf.get_feature_names())
X_train_tfid.head()

X_test_tfid = pd.DataFrame(X_test_tf.todense(), columns=tf.get_feature_names())
X_test_tfid.head()

,aaa,aaaa,aaaaaaaaaaa,aaaaaaaaaahhhhhhhh,aaaaaaaaaamazing,aaaaaaaafternoon,aaaaaaaahhhhhhhh,aaaaaah,aaaaaahhhhhhhh,aaaaaawwwesome,aaaaahhhh,aaaaall,aaaaand,aaaaaoouoouoouu,aaaaarrrrggghhh,aaaaaw,aaaaawwwwwww,aaaagggessss,aaaagh,aaaah,aaaahh,aaaargh,aaaau,aaaaw,aaaawww,aaah,aaahaha,aaahh,aaahhh,aaand,aaargh,aaarrrggghhh,aaarrrgh,aaauuuggghhh,aaaw,aaaww,aaawww,aac,aaggh,aah,...,zenjar,zensify,zero,zeta,zic,zimmer,zion,zip,zipcode,zippys,zit,zoe,zoey,zoidberg,zoita,zojuist,zombie,zombiepix,zomg,zommgg,zone,zoned,zongzi,zoo,zoombezi,zoomed,zoozoo,zotz,zsg,zuccini,zulu,zum,zumba,zune,zwarte,zyrtec,zzzz,zzzzy,zzzzzzz,zzzzzzzzzzzzzzz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
from sklearn.preprocessing import LabelEncoder
out_encoder = LabelEncoder()
out_encoder.fit(y_train)
y_train_tfid = out_encoder.transform(y_train)
y_test_tfid = out_encoder.transform(y_test)

out_encoder.classes_

array(['negative', 'neutral', 'positive'], dtype=object)

In [0]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

naive_params = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
grid_naive = GridSearchCV(MultinomialNB(), naive_params)
grid_naive.fit(X_train_tfid, y_train_tfid)

# SVC best estimator
naive = grid_naive.best_estimator_
print(naive)

naive_score = cross_val_score(naive, X_train_tfid, y_train_tfid, cv=5)
print('Naive modeling using TfidfVectorizer Cross Validation Score: ', round(naive_score.mean() * 100, 2).astype(str) + '%')

y_test_pred =  naive.predict(X_test_tfid)
print("Test accuracy using TfidfVectorizer is:", round(accuracy_score(y_test_tfid,y_test_pred) * 100, 2).astype(str) + '%')

MultinomialNB(alpha=1, class_prior=None, fit_prior=True)
Naive modeling using TfidfVectorizer Cross Validation Score:  62.69%
Test accuracy using TfidfVectorizer is: 63.41%
